## Crossroad

This IPython notebook file seems to be about creating "crossroad polygons" from geospatial street data that is used in **00_C_creating_and_oregenzing_OSM 01**. It involves loading street network data, identifying different shapes like triangles and rectangles in the network, buffering those shapes to make polygons, and filtering the results to get clean crossroad polygons.  

Here are the files it uses:

    csv_tables/TA_streets_20240724_031704/Streets.shp: Geospatial data for streets.

    os_ta_streets_nodes: Node data for the street network.

    os_ta_streets_edges: Edge data for the street network.

    edges_rectangles: Rectangle shapes identified in the street network.

    edges_triangles: Triangle shapes identified in the street network.

    edges_roundabout: Roundabout shapes in the street network.

    edges_rest: Remaining edges in the street network.

    ta_poly_crossroads: Crossroad polygons.

    ta_crossroads_08012025: Processed crossroad data.   

The notebook reads these files, processes the data, and creates polygons by buffering. Along the way, it visualizes the data using maps and also uses tools like leafmap to create interactive maps for better understanding and analysis.

In [308]:
import matplotlib.pyplot as plt
from shapely.geometry import MultiLineString, LineString
from shapely.geometry import Point
from shapely.ops import unary_union
import leafmap

import networkx as nx
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
from IPython.display import Image, display

List of files used:
* csv_tables/TA_streets_20240724_031704/Streets.shp
* os_ta_streets_nodes
* os_ta_streets_edges
* edges_rectangles
* edges_triangles
* edges_roundabout
* edges_rest
* ta_poly_crossroads
* ta_crossroads_08012025 number is date of export

In [309]:
# pd.set_option("display.max_rows", 5)


In [310]:
cols_to_export = ['os_ta_index', 'u', 'v', 'osmid', 'name', 'reversed', 'length'
                  , 'tunnel', 'bridge', 'junction', 'name_type', 'osmid_type',
       'name_fixed', 'ta_name', 'overlapping_names', 'overlapping_osmids',
       'overlapping_names_len', 'start_edge_idx', 'end_edge_idx', 'start_name',
       'end_name']

### Loading relevant data frames

In [311]:
# G = ox.graph_from_place("Tel Aviv, Israel", network_type="drive")
# # you can convert your graph to node and edge GeoPandas GeoDataFrames
# os_ta_streets_nodes, _ = ox.graph_to_gdfs(G)

# os_ta_streets_nodes = os_ta_streets_nodes.to_crs('32636')

# os_ta_streets_nodes = os_ta_streets_nodes.reset_index()
# os_ta_streets_nodes

In [312]:
os_ta_streets_nodes = gpd.read_parquet('./csv_tables/os_ta_streets_nodes.parquet')
os_ta_streets_nodes

,osmid,y,x,highway,street_count,ref,geometry,is_roundabout
0,139693,32.093840,34.790572,traffic_signals,4,None,POINT (668968.683 3552240.237),0
1,139698,32.093869,34.791231,None,3,None,POINT (669030.815 3552244.552),0
2,139707,32.095354,34.778500,None,3,None,POINT (667826.578 3552389.242),0
3,139708,32.095052,34.778329,None,3,None,POINT (667810.983 3552355.494),0
4,139709,32.094527,34.778842,None,4,None,POINT (667860.387 3552298.098),0
...,...,...,...,...,...,...,...,...
6483,12292683832,32.062786,34.785004,None,1,None,POINT (668500.151 3548788.711),0
6484,12292683834,32.063149,34.785276,traffic_signals,3,None,POINT (668525.204 3548829.326),0
6485,12361383714,32.057552,34.763578,None,1,None,POINT (666486.760 3548175.193),0
6486,12361652364,32.054494,34.771553,None,1,None,POINT (667245.361 3547848.419),0


In [313]:
os_ta_streets_nodes.head()


,osmid,y,x,highway,street_count,ref,geometry,is_roundabout
0,139693,32.093840,34.790572,traffic_signals,4,None,POINT (668968.683 3552240.237),0
1,139698,32.093869,34.791231,None,3,None,POINT (669030.815 3552244.552),0
2,139707,32.095354,34.778500,None,3,None,POINT (667826.578 3552389.242),0
3,139708,32.095052,34.778329,None,3,None,POINT (667810.983 3552355.494),0
4,139709,32.094527,34.778842,None,4,None,POINT (667860.387 3552298.098),0


In [314]:
os_ta_streets_edges = gpd.read_parquet('./csv_tables/os_ta_streets_edges.parquet')
os_ta_streets_edges.head(3)

,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,name_fixed,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab
0,0,"LINESTRING (668968.683 3552240.237, 668968.629...",139693,5723720351,5118378,ויצמן,False,4.37,,,...,ויצמן,ויצמן,,,0,1,8725,יהודה המכבי,יהודה המכבי,0
1,1,"LINESTRING (668968.683 3552240.237, 668972.601...",139693,139698,167691710,יהודה המכבי,False,62.173,,,...,יהודה המכבי,יהודה המכבי,,,0,0,2,ויצמן,יהודה המכבי,0
2,2,"LINESTRING (669030.815 3552244.552, 669082.911...",139698,139723,167691710,יהודה המכבי,False,110.029,,,...,יהודה המכבי,יהודה המכבי,,,0,1,23,יהודה המכבי,יהודה המכבי,0


In [315]:
ta_streets = gpd.read_file('./csv_tables/TA_streets_20240724_031704/Streets.shp')
ta_streets

,oidrechov,krechov,trechov,shemangli,mslamas,tsug,kkivun,UniqueId,shemarvit,kreka,geometry
0,1.0,915.0,הרוגי מלכות,HARUGEY MALKHOT,336.0,רחוב,0.0,507-10001,قتل مملكة,100.0,"LINESTRING (672865.880 3554095.253, 672895.216..."
1,2.0,0.0,0,UKNOWN,0.0,רחוב,3.0,507-10002,None,100.0,"LINESTRING (666990.498 3551436.940, 667065.337..."
2,3.0,265.0,אמסטרדם,AMSTERDAM,516.0,רחוב,1.0,507-10003,أمستردام,100.0,"LINESTRING (667879.712 3551424.162, 667940.741..."
3,4.0,644.0,אלון יגאל,YIG'AL ALLON,2524.0,רחוב,0.0,507-10004,ألون ييغال,200.0,"LINESTRING (669570.036 3550420.535, 669581.404..."
4,5.0,634.0,מרגולין,MARGOLIN,2649.0,רחוב,1.0,507-10005,مارغولين,100.0,"LINESTRING (669329.153 3548322.758, 669409.403..."
...,...,...,...,...,...,...,...,...,...,...,...
8874,9851.0,3007.0,שבטי ישראל,SHIVTEY YISRA'EL,1983.0,רחוב,0.0,507-17843,قبائل إسرائيل,100.0,"LINESTRING (665771.816 3547023.159, 665760.256..."
8875,9852.0,3058.0,אבינרי יצחק,AVINERY,2027.0,רחוב,0.0,507-20562,Avinri Yitzhak,100.0,"LINESTRING (665585.719 3547178.152, 665627.936..."
8876,9853.0,3058.0,אבינרי יצחק,AVINERY,2027.0,רחוב,0.0,507-20563,Avinri Yitzhak,100.0,"LINESTRING (665700.142 3547064.296, 665759.119..."
8877,9855.0,3907.0,3907,None,1703.0,רחוב,0.0,507-21960,3907,100.0,"LINESTRING (665087.059 3546677.092, 665075.120..."


In [316]:
os_ta_streets_edges

,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,name_fixed,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab
0,0,"LINESTRING (668968.683 3552240.237, 668968.629...",139693,5723720351,5118378,ויצמן,False,4.37,,,...,ויצמן,ויצמן,,,0,1,8725,יהודה המכבי,יהודה המכבי,0
1,1,"LINESTRING (668968.683 3552240.237, 668972.601...",139693,139698,167691710,יהודה המכבי,False,62.173,,,...,יהודה המכבי,יהודה המכבי,,,0,0,2,ויצמן,יהודה המכבי,0
2,2,"LINESTRING (669030.815 3552244.552, 669082.911...",139698,139723,167691710,יהודה המכבי,False,110.029,,,...,יהודה המכבי,יהודה המכבי,,,0,1,23,יהודה המכבי,יהודה המכבי,0
3,3,"LINESTRING (667826.578 3552389.242, 667810.983...",139707,139708,26516058,ירמיהו,False,37.249,,,...,ירמיהו,ירמיהו הנביא,,,0,4,5,אוסישקין,ירמיהו הנביא,0
4,4,"LINESTRING (667826.578 3552389.242, 667831.045...",139707,10985355495,1183058410,אוסישקין,False,190.227,,,...,אוסישקין,אוסישקין,,,0,3,9372,ירמיהו הנביא,אוסישקין,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9517,9523,"LINESTRING (668470.214 3547868.809, 668469.837...",12287814424,544561044,"[396627330, 396627331, 396627332, 139744099, 1...",איילון דרום,False,1158.0299999999997,,yes,...,איילון דרום,איילון דרום,,,0,93,4375,איילון דרום,חיל השריון,0
9518,9524,"LINESTRING (668470.214 3547868.809, 668467.408...",12287814424,2213119636,1082074722,,False,656.7250000000001,,,...,,"איילון דרום, כביש ירושלים תל אביב",,,0,93,96,איילון דרום,כביש ירושלים תל אביב,0
9519,9525,"LINESTRING (668470.453 3548624.735, 668458.737...",12292683830,1790662878,167691679,המסגר,False,136.733,,,...,המסגר,המסגר,,,0,500,6863,המסגר,המסגר,0
9520,9526,"LINESTRING (668525.204 3548829.326, 668538.677...",12292683834,6145368549,1106792626,הרכב,False,31.335,,,...,הרכב,הרכב,,,0,5582,2535,המסגר,הרכב,0


### Creating Crossroads Polygons

In [317]:
# Helper split edges function

def split_linestrings_by_length(gdf, split_length=5):
    """Split each LineString in a GeoDataFrame into two LineStrings:
    one starting from the start point with the specified length
    and one ending at the endpoint with the specified length."""
    new_rows = []
    
    for i, row in gdf.iterrows():
        geometry = row.geometry
        
        # Check if the geometry is a LineString
        if isinstance(geometry, LineString):
            length = geometry.length
            
            # Ensure the split length is valid
            if split_length * 2 >= length:
                continue  # Skip splitting if split length exceeds the total length
            
            # Compute the two split points
            split_point_start = geometry.interpolate(split_length)
            split_point_end = geometry.interpolate(length - split_length)
            
            # Create two new LineStrings
            first_segment = LineString([geometry.coords[0], split_point_start.coords[0]])
            second_segment = LineString([split_point_end.coords[0], geometry.coords[-1]])
            
            # Append new rows with the original attributes and new geometries
            new_rows.append({**row.to_dict(), "geometry": first_segment})
            new_rows.append({**row.to_dict(), "geometry": second_segment})
    
    # Create a new GeoDataFrame with the split LineStrings
    return gpd.GeoDataFrame(new_rows, crs=gdf.crs)

### trying getting only edges and nodes that are in a triangle shape.

General plan to getting these nodes/edges.

In the edges data frame I have the information of the nodes in columns u, v.<br>

A closed triangle has:
nodes: a, b, c
edges: 1, 2, 3

* group edges by v (end edge)
* the result will be grouped again for edges that their u edge (beginning edge) == v
* and from that result the if there exist an edge that has  


In [318]:
df = os_ta_streets_edges.copy()
df.shape

(9522, 23)

In [319]:
import collections

# "adj" will map each node -> set of its neighbors (undirected)
adj = collections.defaultdict(set)

for row in df.itertuples():
    # row.u, row.v from the GeoDataFrame
    a, b = row.u, row.v

    # If this is truly undirected, add each way:
    adj[a].add(b)
    adj[b].add(a)


In [320]:
triangles = set()

for a in adj:
    # Consider pairs (b,c) of neighbors of a
    for b in adj[a]:
        if b == a:
            continue
        for c in adj[a]:
            if c == a or c <= b:
                # Avoid duplicates, e.g. (b,c) and (c,b)
                continue
            # Check if b and c are directly connected
            if c in adj[b]:
                tri_nodes = tuple(sorted([a, b, c]))
                triangles.add(tri_nodes)

# "triangles" is a set of 3-node tuples: {('a','b','c'), ('x','y','z'), ...}


In [321]:
def get_edges_for_triangle(tri, edges_gdf):
    """Return the rows (edges) in edges_gdf that connect the 3 nodes in tri."""
    a, b, c = tri

    # We want edges for pairs (a,b), (a,c), (b,c), in either orientation
    mask = (
        ((edges_gdf["u"] == a) & (edges_gdf["v"] == b)) |
        ((edges_gdf["u"] == b) & (edges_gdf["v"] == a)) |
        
        ((edges_gdf["u"] == a) & (edges_gdf["v"] == c)) |
        ((edges_gdf["u"] == c) & (edges_gdf["v"] == a)) |
        
        ((edges_gdf["u"] == b) & (edges_gdf["v"] == c)) |
        ((edges_gdf["u"] == c) & (edges_gdf["v"] == b))
    )

    return edges_gdf[mask]


In [322]:
triangle_to_edges = {}

for tri in triangles:
    sub_gdf = get_edges_for_triangle(tri, df)
    # sub_gdf is still a GeoDataFrame, with geometry & CRS
    triangle_to_edges[tri] = sub_gdf


all_rows = []
for tri in triangles:
    sub_gdf = get_edges_for_triangle(tri, df).copy()
    sub_gdf["triangle"] = [tri] * len(sub_gdf)
    all_rows.append(sub_gdf)

df_triangles = pd.concat(all_rows, ignore_index=True)

# Make sure it is recognized as a GeoDataFrame
df_triangles = gpd.GeoDataFrame(df_triangles, geometry="geometry", crs=df.crs)
df_triangles

,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,triangle
0,2538,"LINESTRING (668857.605 3549021.661, 668865.798...",412815718,2380561402,122217871,יצחק שדה,False,14.815,,,...,יצחק שדה 2,,,0,2537,7694,"בית עובד ,נחל איילון",יצחק שדה 2,0,"(2380561370, 2380561402, 412815718)"
1,7694,"LINESTRING (668863.706 3549025.033, 668866.212...",2380561370,2380561402,239569598,יצחק שדה,False,10.527999999999999,,,...,יצחק שדה 2,,,0,6633,2538,יצחק שדה 2,יצחק שדה 2,0,"(2380561370, 2380561402, 412815718)"
2,7695,"LINESTRING (668863.706 3549025.033, 668861.670...",2380561370,412815718,239569599,יצחק שדה,False,7.021,,,...,יצחק שדה 2,,,0,6633,2537,יצחק שדה 2,"בית עובד ,נחל איילון",0,"(2380561370, 2380561402, 412815718)"
3,774,"LINESTRING (666515.138 3547214.210, 666530.849...",318867658,8717879067,31922999,שלבים,False,36.331,,,...,שלבים,,,0,772,7561,היינה היינריך,,0,"(2216978328, 318867658, 8717879067)"
4,7561,"LINESTRING (666544.452 3547198.209, 666537.855...",2216978328,8717879067,941372085,,False,51.507,,,...,"בן צבי, שלבים",,,0,7562,774,בן צבי,שלבים,0,"(2216978328, 318867658, 8717879067)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,5318,"LINESTRING (672216.661 3554787.864, 672218.610...",1164229440,1107315933,25015237,פנחס רוזן,False,41.628,,,...,רוזן פנחס,,,0,4868,1431,רוזן פנחס,שלונסקי אברהם,0,"(1107315933, 1164229440, 1164229505)"
2012,5319,"LINESTRING (672216.661 3554787.864, 672214.712...",1164229440,1164229505,100711593,,False,66.81,,,...,"רוזן פנחס, שלונסקי אברהם",,,0,4868,5053,רוזן פנחס,שלונסקי אברהם,0,"(1107315933, 1164229440, 1164229505)"
2013,3080,"LINESTRING (669319.288 3553275.595, 669315.232...",440061585,2111357737,223401328,חיים לבנון,False,48.186,,,...,לבנון חיים,,,0,457,7293,לבנון חיים,לבנון חיים,0,"(2111357737, 2111357752, 440061585)"
2014,7296,"LINESTRING (669318.992 3553303.548, 669319.288...",2111357752,440061585,201185924,רדינג,False,28.033,,,...,רדינג,,,0,7297,457,,לבנון חיים,0,"(2111357737, 2111357752, 440061585)"


In [323]:
os_ta_streets_nodes_cp = os_ta_streets_nodes.copy()
os_ta_streets_nodes_cp.geometry = os_ta_streets_nodes_cp.buffer(3)

In [324]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)

# m.add_gdf(df_triangles)
# m.add_gdf(os_ta_streets_nodes_cp, fill_colors=['red'])


# m

List of edges to be excluded from the triangle since they are not crossroad:
* remove roundabouts
[6805, 5681, 9404, 9408,7595, 7596, 5981, 5213, 5212,3722, 3729, 3723, 6051, 6037, 6050, 6074, 6059, 6060, 6073, 6055, 3496, 6130, 6118, 6129,
 9253, 6034, 9399, 6104. 4811, 3878, 4812,5811,  8710, 8695, 8711, 9013, 9017, 4188, 4186, 5109, 5109, 5113, 5085, 5097, 5098, 5104, 7816, 7225, 7223
 5636, 1395, 5701, 5700, 5703, 9268, 4024, 4480, 4022, 4481]

**Too much work!**

-------

Let create polygons from our edges. then use the prior method of creating polygons by breaking and buffering
If a resulting polygon is not included in the triangle polygon then we remove these edges.


Plan:
* create triangle polygons
    * add column of tri_idx
    * get area
* Create medium crossroads polygons
    * add cross_idx
    * calculate area
* sjoin between cross_poly and os_ta_streets_edges
    * group by cross_idx
    * get unique os_ta_idx
* sjoin between cross_poly and tri_poly
    * group by tri_idx
    * count how many cross_poly fall in
    * calculate area share

In [325]:
df_tri_cp = df_triangles.copy()

#### create triangle polygons
df_tri_cp.geometry = df_tri_cp.buffer(6)

df_tri_cp_uni = unary_union(df_tri_cp.geometry)
df_tri_cp_uni = gpd.GeoDataFrame(geometry=[df_tri_cp_uni], crs=df_tri_cp.crs)

## explode
df_tri_cp_explode = df_tri_cp_uni.explode(index_parts=True).reset_index(drop=True)
df_tri_cp_explode.reset_index(inplace=True)

## adding idx column to later be used to filter 
df_tri_cp_explode.columns = ['tri_id','geometry']
df_tri_cp_explode['tri_area'] = df_tri_cp_explode.geometry.area

df_tri_cp_explode['tri_id'] = df_tri_cp_explode['tri_id'].apply(lambda x: str(x) + '_tri_id')
display(df_tri_cp_explode), df_tri_cp_explode.shape

#### creating cross road poly
os_ta_streets_edges_cp = os_ta_streets_edges.copy()
os_ta_streets_edges_split = split_linestrings_by_length(os_ta_streets_edges_cp, 6)
os_ta_streets_edges_split.geometry = os_ta_streets_edges_split.buffer(6)

os_ta_streets_edges_uni = unary_union(os_ta_streets_edges_split.geometry)
os_ta_streets_edges_uni = gpd.GeoDataFrame(geometry=[os_ta_streets_edges_uni], crs=os_ta_streets_edges.crs)
## explode
cross_cp_explode = os_ta_streets_edges_uni.explode(index_parts=True).reset_index(drop=True)
cross_cp_explode.reset_index(inplace=True)

## adding idx column to later be used to filter 
cross_cp_explode.columns = ['cr_id','geometry']
cross_cp_explode['cr_area'] = cross_cp_explode.geometry.area

cross_cp_explode['cr_id'] = cross_cp_explode['cr_id'].apply(lambda x: str(x) + '_cr_id')
display(cross_cp_explode), cross_cp_explode.shape

,tri_id,geometry,tri_area
0,0_tri_id,"POLYGON ((664982.831 3545197.962, 664982.038 3...",195.587846
1,1_tri_id,"POLYGON ((665609.535 3545526.135, 665609.588 3...",926.351704
2,2_tri_id,"POLYGON ((664898.317 3545554.923, 664898.513 3...",1582.225711
3,3_tri_id,"POLYGON ((665458.800 3545544.147, 665458.872 3...",628.119004
4,4_tri_id,"POLYGON ((665410.377 3545548.979, 665410.356 3...",1102.913438
...,...,...,...
482,482_tri_id,"POLYGON ((669918.402 3557567.898, 669911.911 3...",3185.737946
483,483_tri_id,"POLYGON ((669509.772 3557628.619, 669510.010 3...",1281.317184
484,484_tri_id,"POLYGON ((669740.875 3557627.187, 669741.553 3...",688.631155
485,485_tri_id,"POLYGON ((669700.945 3557639.775, 669696.384 3...",806.610551


,cr_id,geometry,cr_area
0,0_cr_id,"POLYGON ((665227.766 3545034.839, 665227.297 3...",248.547432
1,1_cr_id,"POLYGON ((665172.306 3545070.751, 665172.300 3...",256.915585
2,2_cr_id,"POLYGON ((665128.917 3545098.534, 665128.871 3...",312.986751
3,3_cr_id,"POLYGON ((665224.575 3545132.051, 665225.046 3...",184.915746
4,4_cr_id,"POLYGON ((665175.387 3545165.416, 665175.540 3...",369.751486
...,...,...,...
4622,4622_cr_id,"POLYGON ((670426.140 3555099.150, 670426.574 3...",314.781310
4623,4623_cr_id,"POLYGON ((670698.740 3555095.330, 670698.694 3...",279.099380
4624,4624_cr_id,"POLYGON ((671532.324 3555096.831, 671532.209 3...",316.198848
4625,4625_cr_id,"POLYGON ((672117.040 3555099.764, 672117.010 3...",184.915746


(None, (4627, 3))

#### sjoin between cross_poly and os_ta_streets_edges
* group by cross_idx
* get unique os_ta_idx

In [326]:
cross_sjoin = gpd.sjoin(cross_cp_explode, os_ta_streets_edges, how='inner', predicate='intersects')
cross_sjoingb_os_id = cross_sjoin.groupby('cr_id').os_ta_index.unique()
# cross_sjoingb_os_id
cross_cp_explode['os_id'] = cross_cp_explode['cr_id'].map(lambda x: cross_sjoingb_os_id[x])
cross_cp_explode

,cr_id,geometry,cr_area,os_id
0,0_cr_id,"POLYGON ((665227.766 3545034.839, 665227.297 3...",248.547432,"[695, 9224]"
1,1_cr_id,"POLYGON ((665172.306 3545070.751, 665172.300 3...",256.915585,"[695, 659]"
2,2_cr_id,"POLYGON ((665128.917 3545098.534, 665128.871 3...",312.986751,"[659, 660, 661]"
3,3_cr_id,"POLYGON ((665224.575 3545132.051, 665225.046 3...",184.915746,[668]
4,4_cr_id,"POLYGON ((665175.387 3545165.416, 665175.540 3...",369.751486,"[660, 668, 669, 667]"
...,...,...,...,...
4622,4622_cr_id,"POLYGON ((670426.140 3555099.150, 670426.574 3...",314.781310,"[859, 860, 861]"
4623,4623_cr_id,"POLYGON ((670698.740 3555095.330, 670698.694 3...",279.099380,"[4363, 4364, 3443]"
4624,4624_cr_id,"POLYGON ((671532.324 3555096.831, 671532.209 3...",316.198848,"[2715, 2714, 2716]"
4625,4625_cr_id,"POLYGON ((672117.040 3555099.764, 672117.010 3...",184.915746,[3278]


<!-- #### sjoin between cross_poly and tri_poly
* group by tri_idx
* count how many cross_poly fall in
* calculate area share -->

In [327]:
from itertools import chain

In [328]:
cr_tri_joined = gpd.sjoin(cross_cp_explode, df_tri_cp_explode, how='inner', predicate='intersects')

cr_tri_gb = cr_tri_joined.groupby('tri_id').agg({
    'cr_id':  lambda g: g.count(),
    'cr_area':  lambda g: g.sum(),
    'os_id': lambda g: list(set(chain.from_iterable(g)))
}).reset_index()


cr_tri_gb.columns = ['tri_id','cr_cnt','cr_area', 'os_id']
df_tri_cp_explode['cr_cnt'] = df_tri_cp_explode['tri_id'].map(lambda x: cr_tri_gb[cr_tri_gb.tri_id == x]['cr_cnt'].values[0])
df_tri_cp_explode['cr_area'] = df_tri_cp_explode['tri_id'].map(lambda x: cr_tri_gb[cr_tri_gb.tri_id == x]['cr_area'].values[0])
df_tri_cp_explode['os_id'] = df_tri_cp_explode['tri_id'].map(lambda x: cr_tri_gb[cr_tri_gb.tri_id == x]['os_id'].values[0])


df_tri_cp_explode

,tri_id,geometry,tri_area,cr_cnt,cr_area,os_id
0,0_tri_id,"POLYGON ((664982.831 3545197.962, 664982.038 3...",195.587846,1,453.741343,"[6800, 670, 662, 6798, 6801, 6802, 6799, 661]"
1,1_tri_id,"POLYGON ((665609.535 3545526.135, 665609.588 3...",926.351704,1,917.941966,"[4264, 8965, 4547, 4267, 8967, 8966]"
2,2_tri_id,"POLYGON ((664898.317 3545554.923, 664898.513 3...",1582.225711,2,1213.170841,"[676, 679, 5680, 5684, 5683, 5682, 5686, 678, ..."
3,3_tri_id,"POLYGON ((665458.800 3545544.147, 665458.872 3...",628.119004,2,1254.037892,"[9403, 6803, 694, 9405, 9408, 693, 9407, 9406,..."
4,4_tri_id,"POLYGON ((665410.377 3545548.979, 665410.356 3...",1102.913438,2,1385.746949,"[9403, 694, 6803, 9405, 9408, 693, 9407, 9406,..."
...,...,...,...,...,...,...
482,482_tri_id,"POLYGON ((669918.402 3557567.898, 669911.911 3...",3185.737946,2,682.488258,"[3220, 2197, 5273, 5274]"
483,483_tri_id,"POLYGON ((669509.772 3557628.619, 669510.010 3...",1281.317184,3,1949.323965,"[9359, 9301, 9357, 9362, 9520, 9355, 9361, 936..."
484,484_tri_id,"POLYGON ((669740.875 3557627.187, 669741.553 3...",688.631155,2,1090.338586,"[9214, 9516, 9430, 9518, 9429, 9431, 9432, 921..."
485,485_tri_id,"POLYGON ((669700.945 3557639.775, 669696.384 3...",806.610551,2,831.682797,"[9433, 9435, 9434, 9436, 9432, 7933]"


In [329]:
df_tri_cp_explode['cr_share'] = df_tri_cp_explode['cr_area'] / df_tri_cp_explode['tri_area']
df_tri_cp_explode['cr_share'] = df_tri_cp_explode['cr_share'].apply(lambda x: round(x, 2))
df_tri_cp_explode

,tri_id,geometry,tri_area,cr_cnt,cr_area,os_id,cr_share
0,0_tri_id,"POLYGON ((664982.831 3545197.962, 664982.038 3...",195.587846,1,453.741343,"[6800, 670, 662, 6798, 6801, 6802, 6799, 661]",2.32
1,1_tri_id,"POLYGON ((665609.535 3545526.135, 665609.588 3...",926.351704,1,917.941966,"[4264, 8965, 4547, 4267, 8967, 8966]",0.99
2,2_tri_id,"POLYGON ((664898.317 3545554.923, 664898.513 3...",1582.225711,2,1213.170841,"[676, 679, 5680, 5684, 5683, 5682, 5686, 678, ...",0.77
3,3_tri_id,"POLYGON ((665458.800 3545544.147, 665458.872 3...",628.119004,2,1254.037892,"[9403, 6803, 694, 9405, 9408, 693, 9407, 9406,...",2.00
4,4_tri_id,"POLYGON ((665410.377 3545548.979, 665410.356 3...",1102.913438,2,1385.746949,"[9403, 694, 6803, 9405, 9408, 693, 9407, 9406,...",1.26
...,...,...,...,...,...,...,...
482,482_tri_id,"POLYGON ((669918.402 3557567.898, 669911.911 3...",3185.737946,2,682.488258,"[3220, 2197, 5273, 5274]",0.21
483,483_tri_id,"POLYGON ((669509.772 3557628.619, 669510.010 3...",1281.317184,3,1949.323965,"[9359, 9301, 9357, 9362, 9520, 9355, 9361, 936...",1.52
484,484_tri_id,"POLYGON ((669740.875 3557627.187, 669741.553 3...",688.631155,2,1090.338586,"[9214, 9516, 9430, 9518, 9429, 9431, 9432, 921...",1.58
485,485_tri_id,"POLYGON ((669700.945 3557639.775, 669696.384 3...",806.610551,2,831.682797,"[9433, 9435, 9434, 9436, 9432, 7933]",1.03


Checking different share to determine what os_id to take and then drop from df_triangle

In [330]:
print('shape of 0.25 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.25].shape)
print('shape of 0.35 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.35].shape)
print('shape of 0.45 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.45].shape)
print('shape of 0.55 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.55].shape)
print('shape of 0.65 below share',df_tri_cp_explode[df_tri_cp_explode.cr_share < 0.65].shape)




shape of 0.25 below share (38, 7)
shape of 0.35 below share (68, 7)
shape of 0.45 below share (85, 7)
shape of 0.55 below share (108, 7)
shape of 0.65 below share (129, 7)


In [331]:
df_tri_cp_explode[(df_tri_cp_explode.cr_share < 1.15) & (df_tri_cp_explode.cr_share >= 1.05)].shape

(24, 7)

In [332]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_tri_cp_explode[(df_tri_cp_explode.cr_share < 1.15) & (df_tri_cp_explode.cr_share >= 1.05)])

# m

In [333]:
# below 0.65 to keep
tri_id_to_keep  =['332_tri_id','326_tri_id', '335_tri_id', '353_tri_id','405_tri_id', '442_tri_id','442_tri_id','424_tri_id','424_tri_id', '467_tri_id',
                  '472_tri_id','481_tri_id','484_tri_id','171_tri_id','20_tri_id', '15_tri_id', '9_tri_id', '293_tri_id','265_tri_id','323_tri_id',
                  '437_tri_id','446_tri_id','418_tri_id','483_tri_id','325_tri_id','501_tri_id','203_tri_id','168_tri_id','108_tri_id','84_tri_id','235_tri_id',
                  '67_tri_id','262_tri_id','337_tri_id','60_tri_id','327_tri_id']

tri_to_maybe_keep = ['46_tri_id']

# between 0.95 and 0.65 to drop
tri_id_to_drop = ['87_tri_id','97_tri_id','162_tri_id','26_tri_id','162_tri_id','225_tri_id','493_tri_id','238_tri_id','341_tri_id','376_tri_id','42_tri_id',
                  '77_tri_id','133_tri_id','222_tri_id','2_tri_id','92_tri_id','141_tri_id','423_tri_id','27_tri_id','59_tri_id','223_tri_id','268_tri_id',
                  '374_tri_id','1_tri_id','86_tri_id','185_tri_id','318_tri_id']

Getting os_id from list of drop and keep and preforming accordingly

In [334]:
os_id_keep_ls = df_tri_cp_explode[df_tri_cp_explode.tri_id.isin(tri_id_to_keep)].os_id.to_list()
os_id_keep_ls = list(set(chain.from_iterable(os_id_keep_ls)))
os_id_keep_ls
print('os_id_keep_ls',len(os_id_keep_ls))

os_id_drop_between_95_65_ls = df_tri_cp_explode[df_tri_cp_explode.tri_id.isin(tri_id_to_drop)].os_id.to_list()
os_id_drop_between_95_65_ls = list(set(chain.from_iterable(os_id_drop_between_95_65_ls)))
os_id_drop_between_95_65_ls
print('os_id_drop_between_95_65_ls',len(os_id_drop_between_95_65_ls))


os_id_drop_ls = df_tri_cp_explode[(df_tri_cp_explode.cr_share < 0.65)].os_id.to_list()
os_id_drop_ls = list(set(chain.from_iterable(os_id_drop_ls)))
os_id_drop_ls
print('os_id_drop_ls len:',len(os_id_drop_ls))


os_id_drop_ls = list(set(os_id_drop_ls + os_id_drop_between_95_65_ls))
# remove from os_id_drop_ls the item from os_id_keep_ls
print('os_id_drop_ls len:',len(os_id_drop_ls))

os_id_drop_ls = list(set(os_id_drop_ls) - set(os_id_keep_ls))
print('os_id_drop_ls len:',len(os_id_drop_ls))


os_id_keep_ls 351
os_id_drop_between_95_65_ls 241
os_id_drop_ls len: 1116
os_id_drop_ls len: 1310
os_id_drop_ls len: 1142


Making sure what I keep and what I drop make sense

In [335]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_triangles[df_triangles.os_ta_index.isin(os_id_drop_ls)])
# m.add_gdf(df_triangles[df_triangles.os_ta_index.isin(os_id_keep_ls)], fill_colors='black')


# m

Dropping from triangle

There is a difference in number of os_id in drop list compared to what is in df_triangles<br>
This is because when creating crossroads we caught edges that weren't in df_triangles. 

In [336]:
df_triangles_clean = df_triangles[~df_triangles.os_ta_index.isin(os_id_drop_ls)]
# df_triangles_clean = df_triangles_clean[]
# df_triangles_clean


In [337]:
df_triangles_clean = df_triangles_clean[(df_triangles_clean.junction != 'roundabout') & (df_triangles_clean.is_connected_to_rab == 0)]

In [338]:
df_triangles_clean.to_parquet('./csv_tables/edges_triangles.parquet')

In [339]:
# st_cnt_2_edges_joined = unary_union(st_cnt_2_edges_buff.geometry)

# # Create polygons GeoDataFrame
# st_cnt_2_edges_joined = gpd.GeoDataFrame(geometry=[st_cnt_2_edges_joined], crs=st_cnt_2_edges_buff.crs)
# st_cnt_2_edges_joined.head(3), st_cnt_2_edges_joined.shape

# st_cnt_2_edges_explode = st_cnt_2_edges_joined.explode(index_parts=True).reset_index(drop=True)
# st_cnt_2_edges_explode.reset_index(inplace=True)


# st_cnt_2_edges_explode.columns = ['st_cnt_2_idx','geometry']

# st_cnt_2_edges_explode['st_cnt_2_idx'] = st_cnt_2_edges_explode['st_cnt_2_idx'].apply(lambda x: str(x) + '_idx')
# display(st_cnt_2_edges_explode), st_cnt_2_edges_explode.shape

# crossroad_st_cnt_2_name = crossroad_st_cnt_2.groupby('st_cnt_2_idx').ta_name.unique()
# crossroad_st_cnt_2_os_ta_index = crossroad_st_cnt_2.groupby('st_cnt_2_idx').os_ta_index.unique()


# crossroad_st_cnt_2_name.name = 'crossroad_name'

# crossroad_st_cnt_2_os_ta_index.name = 'os_edge_idx'

# # crossroad_st_cnt_2_os_ta_index.index.name
# display(crossroad_st_cnt_2_name)
# display(crossroad_st_cnt_2_os_ta_index)

# crossroad_st_cnt_2 = crossroad_st_cnt_2.drop_duplicates(subset=['st_cnt_2_idx'])

# crossroad_st_cnt_2_merged = crossroad_st_cnt_2.merge(crossroad_st_cnt_2_name, on='st_cnt_2_idx')
# crossroad_st_cnt_2_merged = crossroad_st_cnt_2_merged.merge(crossroad_st_cnt_2_os_ta_index, on='st_cnt_2_idx')

# crossroad_st_cnt_2_merged.shape

# def break_name(names):
#     new_names = []
#     for name in names:
#         if ',' in name:
#             split_name = name.split(',')
#             for s_name in split_name:
#                 strip_name = s_name.strip()
#                 new_names.append(strip_name)            
#         else:
#             new_names.append(name)
#     new_names = set(new_names)
#     new_names_ls = list(new_names)
#     return new_names_ls


# crossroad_st_cnt_2_merged.crossroad_name = crossroad_st_cnt_2_merged.crossroad_name.apply(break_name)
# crossroad_st_cnt_2_merged


# def clean_crossroad_name(names):
#     new_names = []
#     for name in names:
#         if len(name) > 1 and name != '':
#             new_names.append(name)
#     return new_names


# crossroad_st_cnt_2_merged.crossroad_name = crossroad_st_cnt_2_merged.crossroad_name.apply(clean_crossroad_name)
# crossroad_st_cnt_2_merged


# crossroad_st_cnt_2_merged['crossroad_name_len'] = crossroad_st_cnt_2_merged.crossroad_name.apply(lambda x: len(x))
# crossroad_st_cnt_2_merged['os_edge_idx_len'] = crossroad_st_cnt_2_merged.os_edge_idx.apply(lambda x: len(x))

# crossroad_st_cnt_2_merged

# crossroad_st_cnt_2_merged['area'] = crossroad_st_cnt_2_merged.area.apply(lambda x: round(x, 3))

#### Getting edges in a shape of a rectangle.

In [340]:
import collections

df = os_ta_streets_edges.copy()

# "adj" will map each node -> set of its neighbors (undirected)
adj = collections.defaultdict(set)

for row in df.itertuples():
    # row.u, row.v from the GeoDataFrame
    a, b = row.u, row.v
    
    # If this is truly undirected, add both directions
    adj[a].add(b)
    adj[b].add(a)


In [341]:
rectangles = set()

for a in adj:
    for b in adj[a]:
        # To avoid re-checking the same pair in reverse
        if b <= a:
            continue

        # Find common neighbors
        common_neighbors = adj[a].intersection(adj[b])
        if len(common_neighbors) < 2:
            continue
        
        # For each pair (c, d) among these neighbors, we form a rectangle
        cn_list = sorted(common_neighbors)
        for i in range(len(cn_list)):
            for j in range(i+1, len(cn_list)):
                c = cn_list[i]
                d = cn_list[j]
                
                # The rectangle is the set of nodes {a, b, c, d}
                # Sort them so we don't double-count the same 4-cycle
                rect_nodes = tuple(sorted([a, b, c, d]))
                rectangles.add(rect_nodes)

print("Number of rectangles found:", len(rectangles))


Number of rectangles found: 42


In [342]:
def get_edges_for_rectangle(rect, edges_gdf):
    """Return the rows (edges) in edges_gdf that connect the 4 nodes in rect."""
    a, b, c, d = rect  # rect is a tuple of 4 node IDs
    
    # We'll collect *all* edges that connect these 4 nodes pairwise.
    # This may return up to 6 edges (the complete subgraph on 4 nodes).
    # If you only want the 4 edges that form the actual cycle,
    # you might need additional logic to pick exactly the cycle edges.
    
    mask = (
        (((edges_gdf["u"] == a) & (edges_gdf["v"] == b)) | 
         ((edges_gdf["u"] == b) & (edges_gdf["v"] == a))) |

        (((edges_gdf["u"] == a) & (edges_gdf["v"] == c)) | 
         ((edges_gdf["u"] == c) & (edges_gdf["v"] == a))) |

        (((edges_gdf["u"] == a) & (edges_gdf["v"] == d)) | 
         ((edges_gdf["u"] == d) & (edges_gdf["v"] == a))) |

        (((edges_gdf["u"] == b) & (edges_gdf["v"] == c)) | 
         ((edges_gdf["u"] == c) & (edges_gdf["v"] == b))) |

        (((edges_gdf["u"] == b) & (edges_gdf["v"] == d)) | 
         ((edges_gdf["u"] == d) & (edges_gdf["v"] == b))) |

        (((edges_gdf["u"] == c) & (edges_gdf["v"] == d)) | 
         ((edges_gdf["u"] == d) & (edges_gdf["v"] == c)))
    )

    return edges_gdf[mask]


In [343]:
rectangle_to_edges = {}

for rect in rectangles:
    sub_gdf = get_edges_for_rectangle(rect, df)
    rectangle_to_edges[rect] = sub_gdf



all_rows = []
for rect in rectangles:
    sub_gdf = get_edges_for_rectangle(rect, df).copy()
    # Tag each row with the rectangle it belongs to
    sub_gdf["rectangle"] = [rect] * len(sub_gdf)
    all_rows.append(sub_gdf)

df_rectangles = pd.concat(all_rows, ignore_index=True)
df_rectangles = gpd.GeoDataFrame(df_rectangles, geometry="geometry", crs=df.crs)

print("df_rectangles shape:", df_rectangles.shape)
df_rectangles.head()


df_rectangles shape: (210, 24)


,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,rectangle
0,1889,"LINESTRING (673242.602 3553996.882, 673239.136...",381533431,10579294317,912644182,ראול ולנברג,False,5.151,,,...,ולנברג ראול,,,0,1890,9281,שלוש משה,921,0,"(10579294317, 10579297244, 1836381515, 381533431)"
1,1890,"LINESTRING (673242.602 3553996.882, 673239.593...",381533431,10579297244,1134836519,921,False,26.959,,,...,שלוש משה,,,0,1889,1891,ולנברג ראול,שלוש משה,0,"(10579294317, 10579297244, 1836381515, 381533431)"
2,6885,"LINESTRING (673250.392 3553987.311, 673242.602...",1836381515,381533431,1134836520,,False,12.352,,,...,"ולנברג ראול, ולנברג ראול",,,0,6886,1889,ולנברג ראול,ולנברג ראול,0,"(10579294317, 10579297244, 1836381515, 381533431)"
3,9281,"LINESTRING (673239.136 3553993.074, 673250.392...",10579294317,1836381515,1134836518,921,False,12.633,,,...,921,,,0,1889,6885,ולנברג ראול,,0,"(10579294317, 10579297244, 1836381515, 381533431)"
4,9287,"LINESTRING (673221.083 3554012.762, 673222.769...",10579297244,10579294317,1134836521,921,False,26.943000000000005,,,...,921,,,0,1890,1889,שלוש משה,ולנברג ראול,0,"(10579294317, 10579297244, 1836381515, 381533431)"


In [344]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(os_ta_streets_nodes_cp)
# m.add_gdf(os_ta_streets_edges)
# m.add_gdf(df_rectangles, fill_colors='black')
# m.add_gdf(df_triangles, fill_colors='black')



# m

Plan:
* create rect polygons
    * add column of rect_idx
    * get area
* Create medium crossroads polygons
    * add cross_idx
    * calculate area
* sjoin between cross_poly and os_ta_streets_edges
    * group by cross_idx
    * get unique os_ta_idx
* sjoin between cross_poly and rect_poly
    * group by rect_idx
    * count how many cross_poly fall in
    * calculate area share

In [345]:
df_rect_cp = df_rectangles.copy()

#### create triangle polygons
df_rect_cp.geometry = df_rect_cp.buffer(6)

df_rect_cp_uni = unary_union(df_rect_cp.geometry)
df_rect_cp_uni = gpd.GeoDataFrame(geometry=[df_rect_cp_uni], crs=df_rect_cp.crs)

## explode
df_rect_cp_explode = df_rect_cp_uni.explode(index_parts=True).reset_index(drop=True)
df_rect_cp_explode.reset_index(inplace=True)

## adding idx column to later be used to filter 
df_rect_cp_explode.columns = ['rect_id','geometry']
df_rect_cp_explode['rect_area'] = df_rect_cp_explode.geometry.area

df_rect_cp_explode['rect_id'] = df_rect_cp_explode['rect_id'].apply(lambda x: str(x) + '_rect_id')
display(df_rect_cp_explode), df_rect_cp_explode.shape


## ALREADY been done
# #### creating cross road poly
# os_ta_streets_edges_cp = os_ta_streets_edges.copy()
# os_ta_streets_edges_split = split_linestrings_by_length(os_ta_streets_edges_cp, 6)
# os_ta_streets_edges_split.geometry = os_ta_streets_edges_split.buffer(6)

# os_ta_streets_edges_uni = unary_union(os_ta_streets_edges_split.geometry)
# os_ta_streets_edges_uni = gpd.GeoDataFrame(geometry=[os_ta_streets_edges_uni], crs=os_ta_streets_edges.crs)
# ## explode
# cross_cp_explode = os_ta_streets_edges_uni.explode(index_parts=True).reset_index(drop=True)
# cross_cp_explode.reset_index(inplace=True)

# ## adding idx column to later be used to filter 
# cross_cp_explode.columns = ['cr_id','geometry']
# cross_cp_explode['cr_area'] = cross_cp_explode.geometry.area

# cross_cp_explode['cr_id'] = cross_cp_explode['cr_id'].apply(lambda x: str(x) + '_cr_id')
# display(cross_cp_explode), cross_cp_explode.shape

,rect_id,geometry,rect_area
0,0_rect_id,"POLYGON ((669776.223 3546761.848, 669776.403 3...",2016.667668
1,1_rect_id,"POLYGON ((666029.118 3547032.445, 666029.049 3...",4444.238925
2,2_rect_id,"POLYGON ((665903.741 3547310.766, 665903.891 3...",688.354852
3,3_rect_id,"POLYGON ((666190.927 3547322.732, 666181.457 3...",1371.932244
4,4_rect_id,"POLYGON ((666659.427 3547529.141, 666659.683 3...",1311.647197
5,5_rect_id,"POLYGON ((665249.349 3547541.264, 665246.772 3...",1729.283377
6,6_rect_id,"POLYGON ((669147.438 3547826.992, 669147.429 3...",1001.909706
7,7_rect_id,"POLYGON ((665705.204 3547961.278, 665705.380 3...",2931.335449
8,8_rect_id,"POLYGON ((669189.239 3548413.487, 669189.267 3...",736.702284
9,9_rect_id,"POLYGON ((667071.772 3548859.085, 667071.798 3...",2401.537096


(None, (34, 3))

#### sjoin between cross_poly and os_ta_streets_edges
* group by cross_idx
* get unique os_ta_idx

In [346]:
# Already been done

# cross_sjoin = gpd.sjoin(cross_cp_explode, os_ta_streets_edges, how='inner', predicate='intersects')
# cross_sjoingb_os_id = cross_sjoin.groupby('cr_id').os_ta_index.unique()
# # cross_sjoingb_os_id
# cross_cp_explode['os_id'] = cross_cp_explode['cr_id'].map(lambda x: cross_sjoingb_os_id[x])
# cross_cp_explode

In [347]:
cr_rect_joined = gpd.sjoin(cross_cp_explode, df_rect_cp_explode, how='inner', predicate='intersects')

cr_rect_gb = cr_rect_joined.groupby('rect_id').agg({
    'cr_id':  lambda g: g.count(),
    'cr_area':  lambda g: g.sum(),
    'os_id': lambda g: list(set(chain.from_iterable(g)))
}).reset_index()


cr_rect_gb.columns = ['rect_id','cr_cnt','cr_area', 'os_id']
df_rect_cp_explode['cr_cnt'] = df_rect_cp_explode['rect_id'].map(lambda x: cr_rect_gb[cr_rect_gb.rect_id == x]['cr_cnt'].values[0])
df_rect_cp_explode['cr_area'] = df_rect_cp_explode['rect_id'].map(lambda x: cr_rect_gb[cr_rect_gb.rect_id == x]['cr_area'].values[0])
df_rect_cp_explode['os_id'] = df_rect_cp_explode['rect_id'].map(lambda x: cr_rect_gb[cr_rect_gb.rect_id == x]['os_id'].values[0])


df_rect_cp_explode

,rect_id,geometry,rect_area,cr_cnt,cr_area,os_id
0,0_rect_id,"POLYGON ((669776.223 3546761.848, 669776.403 3...",2016.667668,2,970.004823,"[9017, 9011, 9008, 9016, 9015, 9019, 9014, 901..."
1,1_rect_id,"POLYGON ((666029.118 3547032.445, 666029.049 3...",4444.238925,4,1269.373828,"[6937, 6938, 6953, 6946, 6947, 6936, 6948, 6929]"
2,2_rect_id,"POLYGON ((665903.741 3547310.766, 665903.891 3...",688.354852,2,872.484133,"[9149, 3325, 9150, 9151, 5466, 9409, 9148, 342..."
3,3_rect_id,"POLYGON ((666190.927 3547322.732, 666181.457 3...",1371.932244,3,1617.739332,"[760, 762, 759, 7568, 3331, 3695, 3332, 1680, ..."
4,4_rect_id,"POLYGON ((666659.427 3547529.141, 666659.683 3...",1311.647197,2,1154.218159,"[1698, 1697, 6562, 6560, 1701, 5942, 9069, 656..."
5,5_rect_id,"POLYGON ((665249.349 3547541.264, 665246.772 3...",1729.283377,3,2084.651726,"[8391, 3870, 1773, 1764, 3871, 5449, 1676, 177..."
6,6_rect_id,"POLYGON ((669147.438 3547826.992, 669147.429 3...",1001.909706,2,1284.908977,"[5844, 4022, 5717, 9266, 9268, 4024, 5845, 514..."
7,7_rect_id,"POLYGON ((665705.204 3547961.278, 665705.380 3...",2931.335449,4,1207.150337,"[4329, 4327, 1782, 4328, 4325, 8300, 4326]"
8,8_rect_id,"POLYGON ((669189.239 3548413.487, 669189.267 3...",736.702284,1,1729.140057,"[7644, 7141, 5904, 1447, 8078, 5905, 7647, 678..."
9,9_rect_id,"POLYGON ((667071.772 3548859.085, 667071.798 3...",2401.537096,2,1285.875633,"[269, 2063, 273, 8414, 8434, 8436, 2062, 8435,..."


In [348]:
df_rect_cp_explode['cr_share'] = df_rect_cp_explode['cr_area'] / df_rect_cp_explode['rect_area']
df_rect_cp_explode['cr_share'] = df_rect_cp_explode['cr_share'].apply(lambda x: round(x, 2))
df_rect_cp_explode

,rect_id,geometry,rect_area,cr_cnt,cr_area,os_id,cr_share
0,0_rect_id,"POLYGON ((669776.223 3546761.848, 669776.403 3...",2016.667668,2,970.004823,"[9017, 9011, 9008, 9016, 9015, 9019, 9014, 901...",0.48
1,1_rect_id,"POLYGON ((666029.118 3547032.445, 666029.049 3...",4444.238925,4,1269.373828,"[6937, 6938, 6953, 6946, 6947, 6936, 6948, 6929]",0.29
2,2_rect_id,"POLYGON ((665903.741 3547310.766, 665903.891 3...",688.354852,2,872.484133,"[9149, 3325, 9150, 9151, 5466, 9409, 9148, 342...",1.27
3,3_rect_id,"POLYGON ((666190.927 3547322.732, 666181.457 3...",1371.932244,3,1617.739332,"[760, 762, 759, 7568, 3331, 3695, 3332, 1680, ...",1.18
4,4_rect_id,"POLYGON ((666659.427 3547529.141, 666659.683 3...",1311.647197,2,1154.218159,"[1698, 1697, 6562, 6560, 1701, 5942, 9069, 656...",0.88
5,5_rect_id,"POLYGON ((665249.349 3547541.264, 665246.772 3...",1729.283377,3,2084.651726,"[8391, 3870, 1773, 1764, 3871, 5449, 1676, 177...",1.21
6,6_rect_id,"POLYGON ((669147.438 3547826.992, 669147.429 3...",1001.909706,2,1284.908977,"[5844, 4022, 5717, 9266, 9268, 4024, 5845, 514...",1.28
7,7_rect_id,"POLYGON ((665705.204 3547961.278, 665705.380 3...",2931.335449,4,1207.150337,"[4329, 4327, 1782, 4328, 4325, 8300, 4326]",0.41
8,8_rect_id,"POLYGON ((669189.239 3548413.487, 669189.267 3...",736.702284,1,1729.140057,"[7644, 7141, 5904, 1447, 8078, 5905, 7647, 678...",2.35
9,9_rect_id,"POLYGON ((667071.772 3548859.085, 667071.798 3...",2401.537096,2,1285.875633,"[269, 2063, 273, 8414, 8434, 8436, 2062, 8435,...",0.54


Checking different share to determine what os_id to take and then drop from df_triangle

In [349]:
print('shape of 0.25 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.25].shape)
print('shape of 0.35 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.35].shape)
print('shape of 0.45 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.45].shape)
print('shape of 0.55 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.55].shape)
print('shape of 0.65 below share',df_rect_cp_explode[df_rect_cp_explode.cr_share < 0.65].shape)




shape of 0.25 below share (2, 7)
shape of 0.35 below share (3, 7)
shape of 0.45 below share (5, 7)
shape of 0.55 below share (9, 7)
shape of 0.65 below share (9, 7)


In [350]:
df_rect_cp_explode[(df_rect_cp_explode.cr_share < 1.15) & (df_rect_cp_explode.cr_share >= 1.05)].shape

(1, 7)

In [351]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_rect_cp_explode)

# m

In [352]:
rect_id_to_keep  =[]

rect_to_maybe_keep = []

rect_id_to_drop = ['1_rect_id', '7_rect_id','9_rect_id','14_rect_id','18_rect_id','27_rect_id']

Getting os_id from list of drop and keep and preforming accordingly

In [353]:
os_id_drop_ls = df_rect_cp_explode[df_rect_cp_explode.rect_id.isin(rect_id_to_drop)].os_id.to_list()
os_id_drop_ls = list(set(chain.from_iterable(os_id_drop_ls)))
os_id_drop_ls
print('os_id_drop_ls',len(os_id_drop_ls))

os_id_drop_ls 65


Making sure what I keep and what I drop make sense

In [354]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# m.add_gdf(df_rectangles[df_rectangles.os_ta_index.isin(os_id_drop_ls)])

# m

Dropping from triangle

There is a difference in number of os_id in drop list compared to what is in df_triangles<br>
This is because when creating crossroads we caught edges that weren't in def_triangles. 

In [355]:
df_rectangles_clean = df_rectangles[~df_rectangles.os_ta_index.isin(os_id_drop_ls)]


In [356]:
df_rectangles_clean = df_rectangles_clean[(df_rectangles_clean.junction != 'roundabout') & (df_rectangles_clean.is_connected_to_rab == 0)]

In [357]:
df_rectangles_clean.to_parquet('./csv_tables/edges_rectangles.parquet')

### Exporting roundabout data
We already have this so we can simply export

In [358]:
rab_to_fix_idx = os_ta_streets_edges[(os_ta_streets_edges.junction == 'roundabout') & (os_ta_streets_edges.is_connected_to_rab == 1)].index
os_ta_streets_edges.loc[rab_to_fix_idx, 'is_connected_to_rab'] = 0

In [359]:
os_ta_streets_edges.to_parquet('./csv_tables/os_ta_streets_edges.parquet')

In [360]:
ra_cr = os_ta_streets_edges[(os_ta_streets_edges.junction == 'roundabout')].copy()
print(ra_cr.shape)
ra_cr.head(3)

(515, 23)


,os_ta_index,geometry,u,v,osmid,name,reversed,length,tunnel,bridge,...,name_fixed,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab
177,177,"LINESTRING (668874.869 3551329.236, 668885.399...",35288627,2203627170,5118376,ה' באייר,False,27.759999999999998,,,...,ה' באייר,ה' באייר,"הא באייר ,ז'בוטינסקי ,ויצמן ,חברה חדשה ,תש״ח ,...",5118376,6,178,1284,ויצמן,ויצמן,0
256,256,"LINESTRING (672928.438 3554590.534, 672933.715...",271878297,381571872,33492500,כיכר ומשמר הירדן,False,28.725,,,...,,כיכר ומשמר הירדן,",משמר הירדן ,מרכוס דוד ,פתחיה מרגנשבורג","1134738136 ,33492500",4,257,1900,משמר הירדן,מרכוס דוד,0
258,258,"LINESTRING (672940.500 3554645.491, 672933.864...",271878300,1170469323,33492500,כיכר ומשמר הירדן,False,15.693,,,...,,כיכר ומשמר הירדן,",משמר הירדן ,מרכוס דוד ,פתחיה מרגנשבורג","1134738136 ,33492500",4,251,2745,משמר הירדן,פתחיה מרגנשבורג,0


In [361]:
ra_cr.to_parquet('./csv_tables/edges_roundabout.parquet')

In [362]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# # m.add_gdf(ra_cr)
# m.add_gdf( os_ta_streets_edges[(os_ta_streets_edges.is_connected_to_rab == 1)], fill_colors='black')
# m.add_gdf(ra_cr)



# m

### Looking at Roundabout Triangle and rectangle Crossroads

In [363]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')


In [364]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)


# # m.add_gdf(ra_cr)
# m.add_gdf(tri_cr)
# m.add_gdf(rect_cr, fill_colors='black')
# # m.add_gdf(ra_cr)

# m

### Dropping edges that exist in tri and rect and exporting again

In [365]:
rect_os_ta_index = rect_edges.os_ta_index.to_list()
os_ta_index_to_filter = tri_edges[tri_edges.os_ta_index.isin(rect_os_ta_index)].os_ta_index

rect_edges = rect_edges[~rect_edges.os_ta_index.isin(os_ta_index_to_filter)].copy()

rect_edges.to_parquet('./csv_tables/edges_rectangles.parquet')

In [366]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(tri_edges[tri_edges.os_ta_index.isin(rect_os_ta_index)], fill_colors='black')

# m

###  Creating edges that not in rect, tri, and ra

In [367]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')


In [368]:
os_ta_streets_edges[(~os_ta_streets_edges.os_ta_index.isin(rect_edges.os_ta_index)) & 
                    (~os_ta_streets_edges.os_ta_index.isin(tri_edges.os_ta_index)) & 
                    (~os_ta_streets_edges.os_ta_index.isin(ra_edges.os_ta_index))].to_parquet('./csv_tables/edges_rest.parquet')

## Creating polygons from tri, rect, ra and rest

In [369]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')
rest_edges = gpd.read_parquet('./csv_tables/edges_rest.parquet')

### rect polygon

In [370]:
# Buffer
rect_edges_cp = rect_edges.copy()
rect_edges_cp.geometry = rect_edges_cp.buffer(5)

rect_edges_uni =  unary_union(rect_edges_cp.geometry)

# gdf
rect_edges_uni_gdf = gpd.GeoDataFrame(geometry=[rect_edges_uni], crs=rect_edges_cp.crs)
rect_edges_poly_gdf = rect_edges_uni_gdf.explode(index_parts=True).reset_index(drop=True)
rect_edges_poly_gdf.geometry = rect_edges_poly_gdf.convex_hull

### tri polygon

In [371]:
# Buffer
tri_edges_cp = tri_edges.copy()
tri_edges_cp.geometry = tri_edges_cp.buffer(5)

tri_edges_uni =  unary_union(tri_edges_cp.geometry)

# gdf
tri_edges_uni_gdf = gpd.GeoDataFrame(geometry=[tri_edges_uni], crs=tri_edges_cp.crs)
tri_edges_poly_gdf = tri_edges_uni_gdf.explode(index_parts=True).reset_index(drop=True)
tri_edges_poly_gdf
tri_edges_poly_gdf.geometry = tri_edges_poly_gdf.convex_hull

### ra polygon

In [372]:
# Buffer
ra_edges_cp = ra_edges.copy()
ra_edges_cp.geometry = ra_edges_cp.buffer(5)

ra_edges_uni =  unary_union(ra_edges_cp.geometry)

# gdf
ra_edges_uni_gdf = gpd.GeoDataFrame(geometry=[ra_edges_uni], crs=ra_edges_cp.crs)
ra_edges_poly_gdf = ra_edges_uni_gdf.explode(index_parts=True).reset_index(drop=True)
ra_edges_poly_gdf.geometry = ra_edges_poly_gdf.convex_hull

In [373]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(tri_edges_poly_gdf)
# # m.add_gdf(tri_edges,fill_colors='black')


# m

## Creating Crossroads and checking how they look

* create crossroads using edges_rest with:
    * rect_edges
    * tri_edges 
    * ra_edges
    * view on map

plan:
* split edges_rest
* buffer edges_rest
* union with buffed rect/tri/ra/edges
* explode to single polygons
* view on map

If good:
* sjoin the result on os_ta_edges
* name each crossroad
* export 


In [374]:
rect_edges = gpd.read_parquet('./csv_tables/edges_rectangles.parquet')
tri_edges  = gpd.read_parquet('./csv_tables/edges_triangles.parquet')
ra_edges   = gpd.read_parquet('./csv_tables/edges_roundabout.parquet')
rest_edges = gpd.read_parquet('./csv_tables/edges_rest.parquet')

In [375]:
# split edges_rest
rest_edges_split = split_linestrings_by_length( rest_edges, 6)
rest_edges_split.shape

# buffer edges_rest
rest_edges_split.geometry = rest_edges_split.buffer(5.5)
rest_edges_split


# union with rect/tri/ra
all_geoms_ls = rest_edges_split.geometry.to_list() + rect_edges_poly_gdf.geometry.to_list() + tri_edges_poly_gdf.geometry.to_list() + ra_edges_poly_gdf.geometry.to_list()
all_geoms = gpd.GeoSeries(all_geoms_ls)
# # len(rest_and_rect_geoms)
all_geoms_uni = unary_union(all_geoms)
all_geoms_gdf = gpd.GeoDataFrame(geometry=[all_geoms_uni], crs=rest_edges.crs)
all_geoms_poly_gdf = all_geoms_gdf.explode(index_parts=True).reset_index(drop=True)




In [376]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(all_geoms_poly_gdf)

# m

In [377]:
all_geoms_poly_gdf.to_parquet('./csv_tables/ta_poly_crossroads.parquet')

### conclusion of creating geoms:

I think I have reached a point where trying to get better results is not worth the efforts. 

What are some of the issues **all_geoms_poly_gdf** currently has:
* some streets in florentin are connected while they shouldn't<br>
  This is because of how small and short these streets are.
* some area like **montifiory** are not connected well even though they should

There are more places with this kind of issue.

My next task is:
* Add area to crossroads
* Add an identifier
* combine the data from os_ta_streets_edges with the all_geoms_poly_gdf.
* Give each polygon/crossroad a name
* add number of edges each polygon is constructed
* Add which edges it is constructed
* Add which nodes it is constructed


In [378]:
all_geoms_poly_gdf = gpd.read_parquet('./csv_tables/ta_poly_crossroads.parquet')

#### Add area to crossroads

In [379]:
all_geoms_poly_gdf['cr_area'] = all_geoms_poly_gdf.area

#### Give each polygon/crossroad a name and which edges it is constructed

In [380]:
all_geoms_poly_gdf.reset_index(inplace=True)

In [381]:
all_geoms_poly_gdf.columns

Index(['index', 'geometry', 'cr_area'], dtype='object')

In [382]:
all_geoms_poly_gdf.columns = ['cr_idx','geometry','cr_area']

Connect os_ta_edges with all_geoms_poly_gdf

In [383]:
ta_edges_all_cr_joined = gpd.sjoin(all_geoms_poly_gdf, os_ta_streets_edges, how='inner', predicate='intersects')

ta_edges_all_cr_joined.head(3)

,cr_idx,geometry,cr_area,index_right,os_ta_index,u,v,osmid,name,reversed,...,name_fixed,ta_name,overlapping_names,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab
0,0,"POLYGON ((665228.049 3545035.251, 665227.620 3...",219.847292,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,הגבול (שמחה הולצברג),הגבול (שמחה הולצברג),,,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0
0,0,"POLYGON ((665228.049 3545035.251, 665227.620 3...",219.847292,9218,9224,9971016122,318180661,1124797593,שדרות ירושלים,False,...,שדרות ירושלים,שדרות ירושלים,,,0,693,695,שדרות ירושלים,הגבול (שמחה הולצברג),0
1,1,"POLYGON ((665172.574 3545071.173, 665172.569 3...",226.880445,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,הגבול (שמחה הולצברג),הגבול (שמחה הולצברג),,,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0


In [384]:
def get_cr_name(arr):
    arr_stripped = [s.strip() for s in arr]
    arr_sorted = sorted(arr_stripped)
    u_ls = list(set(arr_sorted))
    str_j = ' ,'.join(u_ls)
    return str_j

In [385]:
ta_joined_gb_ta_name = ta_edges_all_cr_joined.groupby('cr_idx').agg({'ta_name':get_cr_name, 'os_ta_index':pd.unique}) 

ta_joined_gb_ta_name

,ta_name,os_ta_index
cr_idx,,
0,"שדרות ירושלים ,הגבול (שמחה הולצברג)","[695, 9224]"
1,הגבול (שמחה הולצברג),"[695, 659]"
2,"קרוא ברוך ,הגבול (שמחה הולצברג)","[659, 660, 661]"
3,מייזל זלמן,[668]
4,"קרוא ברוך ,מייזל זלמן","[660, 668, 669, 667]"
...,...,...
4277,"כיכר ו2170 ,יוניצ'מן ,2170","[1861, 9437, 1862, 5173]"
4278,"דרך תל-אביב, ,דרך תל-אביב","[5274, 5273]"
4279,2170,[5173]


In [386]:
ta_edges_all_cr_joined['cr_name'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'ta_name'])
ta_edges_all_cr_joined['cr_edges'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: ta_joined_gb_ta_name.loc[x, 'os_ta_index'])


#### Adding which nodes are related to crossroads 

In [387]:
ta_nodes_all_cr_joined = gpd.sjoin(os_ta_streets_nodes, all_geoms_poly_gdf, how='inner', predicate='intersects')

ta_nodes_all_cr_joined.head(3)

,osmid,y,x,highway,street_count,ref,geometry,is_roundabout,index_right,cr_idx,cr_area
0,139693,32.093840,34.790572,traffic_signals,4,None,POINT (668968.683 3552240.237),0,3217,3217,644.868272
1,139698,32.093869,34.791231,None,3,None,POINT (669030.815 3552244.552),0,3219,3219,280.047820
2,139707,32.095354,34.778500,None,3,None,POINT (667826.578 3552389.242),0,3269,3269,282.140195


In [388]:
nodes_cr_gb = ta_nodes_all_cr_joined.groupby('cr_idx').osmid.unique()
nodes_cr_gb

cr_idx
0                    [318180661]
1                    [318177898]
2                    [318177899]
3                    [318178557]
4                    [318178468]
                  ...           
4277    [374352685, 11267492233]
4278                [1144043820]
4279                [1131244786]
4280                [9931532351]
4281                [2868373851]
Name: osmid, Length: 4282, dtype: object

#### Adding to ta_edges_all_cr_joined the nodes related to cr

In [389]:
ta_edges_all_cr_joined['cr_nodes'] = ta_edges_all_cr_joined.cr_idx.map(lambda x: nodes_cr_gb[x])

In [390]:
ta_edges_all_cr_joined.loc[0, 'cr_name']

0    שדרות ירושלים ,הגבול (שמחה הולצברג)
0    שדרות ירושלים ,הגבול (שמחה הולצברג)
Name: cr_name, dtype: object

In [391]:
ta_edges_all_cr_joined.head(3)

,cr_idx,geometry,cr_area,index_right,os_ta_index,u,v,osmid,name,reversed,...,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,cr_name,cr_edges,cr_nodes
0,0,"POLYGON ((665228.049 3545035.251, 665227.620 3...",219.847292,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0,"שדרות ירושלים ,הגבול (שמחה הולצברג)","[695, 9224]",[318180661]
0,0,"POLYGON ((665228.049 3545035.251, 665227.620 3...",219.847292,9218,9224,9971016122,318180661,1124797593,שדרות ירושלים,False,...,,0,693,695,שדרות ירושלים,הגבול (שמחה הולצברג),0,"שדרות ירושלים ,הגבול (שמחה הולצברג)","[695, 9224]",[318180661]
1,1,"POLYGON ((665172.574 3545071.173, 665172.569 3...",226.880445,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0,הגבול (שמחה הולצברג),"[695, 659]",[318177898]


In [392]:
ta_edges_all_cr_joined.columns

Index(['cr_idx', 'geometry', 'cr_area', 'index_right', 'os_ta_index', 'u', 'v',
       'osmid', 'name', 'reversed', 'length', 'tunnel', 'bridge', 'junction',
       'name_type', 'osmid_type', 'name_fixed', 'ta_name', 'overlapping_names',
       'overlapping_osmids', 'overlapping_names_len', 'start_edge_idx',
       'end_edge_idx', 'start_name', 'end_name', 'is_connected_to_rab',
       'cr_name', 'cr_edges', 'cr_nodes'],
      dtype='object')

In [393]:
cols_to_keep = ['geometry','cr_idx','cr_name','cr_edges','cr_nodes']

In [394]:
ta_edges_all_cr_joined_no_dup = ta_edges_all_cr_joined.drop_duplicates(subset=['cr_idx','cr_name'])
ta_edges_all_cr_joined_no_dup[cols_to_keep]

,geometry,cr_idx,cr_name,cr_edges,cr_nodes
0,"POLYGON ((665228.049 3545035.251, 665227.620 3...",0,"שדרות ירושלים ,הגבול (שמחה הולצברג)","[695, 9224]",[318180661]
1,"POLYGON ((665172.574 3545071.173, 665172.569 3...",1,הגבול (שמחה הולצברג),"[695, 659]",[318177898]
2,"POLYGON ((665129.187 3545098.955, 665129.145 3...",2,"קרוא ברוך ,הגבול (שמחה הולצברג)","[659, 660, 661]",[318177899]
3,"POLYGON ((665224.294 3545131.637, 665224.726 3...",3,מייזל זלמן,[668],[318178557]
4,"POLYGON ((665174.710 3545165.254, 665175.272 3...",4,"קרוא ברוך ,מייזל זלמן","[660, 668, 669, 667]",[318178468]
...,...,...,...,...,...
4277,"POLYGON ((669263.552 3557726.504, 669264.051 3...",4277,"כיכר ו2170 ,יוניצ'מן ,2170","[1861, 9437, 1862, 5173]","[374352685, 11267492233]"
4278,"POLYGON ((669963.496 3557730.701, 669963.347 3...",4278,"דרך תל-אביב, ,דרך תל-אביב","[5274, 5273]",[1144043820]
4279,"POLYGON ((669028.902 3557743.422, 669029.101 3...",4279,2170,[5173],[1131244786]
4280,"POLYGON ((669862.404 3557808.817, 669862.125 3...",4280,"2379, ,2379","[9215, 9214, 7932]",[9931532351]


In [395]:
ta_edges_all_cr_joined_no_dup[cols_to_keep].to_parquet('./csv_tables/ta_crossroads_08012025.parquet')

In [396]:
ta_edges_all_cr_joined_no_dup

,cr_idx,geometry,cr_area,index_right,os_ta_index,u,v,osmid,name,reversed,...,overlapping_osmids,overlapping_names_len,start_edge_idx,end_edge_idx,start_name,end_name,is_connected_to_rab,cr_name,cr_edges,cr_nodes
0,0,"POLYGON ((665228.049 3545035.251, 665227.620 3...",219.847292,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0,"שדרות ירושלים ,הגבול (שמחה הולצברג)","[695, 9224]",[318180661]
1,1,"POLYGON ((665172.574 3545071.173, 665172.569 3...",226.880445,695,695,318180661,318177898,1124797596,הגבול (שמחה הולצברג),False,...,,0,9224,659,שדרות ירושלים,הגבול (שמחה הולצברג),0,הגבול (שמחה הולצברג),"[695, 659]",[318177898]
2,2,"POLYGON ((665129.187 3545098.955, 665129.145 3...",279.492627,659,659,318177898,318177899,1124797596,הגבול (שמחה הולצברג),False,...,,0,695,660,הגבול (שמחה הולצברג),קרוא ברוך,0,"קרוא ברוך ,הגבול (שמחה הולצברג)","[659, 660, 661]",[318177899]
3,3,"POLYGON ((665224.294 3545131.637, 665224.726 3...",160.880592,668,668,318178468,318178557,654580347,זלמן מייזל,False,...,,0,660,9224,קרוא ברוך,שדרות ירושלים,0,מייזל זלמן,[668],[318178557]
4,4,"POLYGON ((665174.710 3545165.254, 665175.272 3...",332.689221,660,660,318177899,318178468,28941433,ברוך קרוא,False,...,,0,659,667,הגבול (שמחה הולצברג),קרוא ברוך,0,"קרוא ברוך ,מייזל זלמן","[660, 668, 669, 667]",[318178468]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4277,4277,"POLYGON ((669263.552 3557726.504, 669264.051 3...",663.361250,1861,1861,374352685,10901313426,1173228629,יוניצמן,False,...,,0,1862,9348,כיכר ו2170,יוניצ'מן,1,"כיכר ו2170 ,יוניצ'מן ,2170","[1861, 9437, 1862, 5173]","[374352685, 11267492233]"
4278,4278,"POLYGON ((669963.496 3557730.701, 669963.347 3...",164.100399,5273,5274,1144043820,469933508,215840485,דרך תל-אביב,False,...,,0,5273,2197,,,0,"דרך תל-אביב, ,דרך תל-אביב","[5274, 5273]",[1144043820]
4279,4279,"POLYGON ((669028.902 3557743.422, 669029.101 3...",160.880592,5172,5173,1131244786,11267492233,35102674,2170,True,...,,0,9437,1862,כיכר ו2170,כיכר ו2170,1,2170,[5173],[1131244786]
4280,4280,"POLYGON ((669862.404 3557808.817, 669862.125 3...",277.332261,9209,9215,9931532351,12120460109,"[1083754714, 1308836991]",,"[False, True]",...,,0,7932,9516,2379,,0,"2379, ,2379","[9215, 9214, 7932]",[9931532351]


In [ ]:
# ## checking created polygons

# m = leafmap.Map(center=(32.047, 34.785), zoom=11)
# m.add_gdf(all_geoms_poly_gdf)

# m